In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
from datetime import datetime
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRFRegressor
from sklearn.ensemble import RandomForestRegressor
import seaborn as sns
from sklearn.model_selection import GridSearchCV

In [ ]:
filename = '/kaggle/input/real-time-advertisers-auction/Dataset.csv'
df = pd.read_csv(filename, parse_dates=[0])
df.head()

In [ ]:
def weird_division(n, d):
    return n / d if d else 0

In [ ]:
df['CPM'] = df.apply(lambda x: weird_division(((x['total_revenue']*100)),x['measurable_impressions'])*1000 , axis=1)
df['View/measurable'] = df.apply(lambda x: weird_division(x['viewable_impressions'],x['measurable_impressions']) , axis=1)

In [ ]:
df.drop(columns=['revenue_share_percent', 'integration_type_id', 'total_impressions'], inplace=True)

In [ ]:
#Split for train & test
train_df = df[df.date <= datetime(2019, 6, 21)]
test_df = df[df.date > datetime(2019, 6, 21)]
train_df = train_df[train_df.CPM <= train_df.CPM.quantile(0.95)]
test_df = test_df[test_df.CPM <= test_df.CPM.quantile(0.95)]
test_df = test_df[test_df.CPM >= 0]
train_df.reset_index(inplace=True)
test_df.reset_index(inplace=True)

In [ ]:
def get_x_y(frame):
    X = frame[['site_id', 'ad_type_id', 'geo_id', 'device_category_id', 
               'advertiser_id', 'order_id', 'line_item_type_id', 'os_id',
               'monetization_channel_id', 'ad_unit_id', 'viewable_impressions',
               'measurable_impressions', 'View/measurable']]
    y = frame['CPM']
    return X, y

In [ ]:
def fix_preds(X_test, preds):
    for i in X_test.index:
        if X_test.iloc[i]['measurable_impressions'] == 0:
            preds[i] = 0
    for i in range(len(preds)):
        if preds[i] < 0:
            preds[i] = 0
    return preds

In [ ]:
X_train, y_train = get_x_y(train_df)
X_test, y_test = get_x_y(test_df)

In [ ]:
regressors = {'LGBM': LGBMRegressor(objective='mse', n_jobs=-1),
               'CatBoost': CatBoostRegressor(objective='RMSE', verbose=0),
               'XGB': XGBRFRegressor(objective='reg:squarederror'),
               'RandomForest': RandomForestRegressor(criterion='mse', n_jobs=-1)
              }

In [ ]:
for name, regressor in regressors.items():
    regressor.fit(X_train, y_train)
    preds = regressor.predict(X_test)
    print(f'name: {name}, score: {mean_squared_error(y_test, preds)}')

In [ ]:
#After some experiments with GridSearch
regr = LGBMRegressor(metric='mse', n_estimators=300, n_jobs=-1, num_leaves=141, objective='mse')
regr.fit(X_train, y_train)
preds = regr.predict(X_test)
print(mean_squared_error(y_test, preds))
preds = fix_preds(X_test, preds)
print(mean_squared_error(y_test, preds))

### Best Score: 2647
# Hochu Zachet!